In [1]:
# | default_exp backend.llms.response_processor

In [5]:
# | export 
import json
from typing import Dict, Any, Optional
from rich.console import Console
from rich.markdown import Markdown
from agentic.configs.loader import get_reasoning_config
import re

In [3]:
# | export
RESET = "\033[0m"
# Get reasoning config
reasoning_config = get_reasoning_config()
show_thinking = reasoning_config.get('show_thinking', True)
thinking_color = reasoning_config.get('thinking_color', 'pink')

# Set color based on config with enhanced colors
color_codes = {
            'pink': "\033[38;2;200;100;120m",     # Darker pink
            'blue': "\033[38;2;70;130;180m",      # Steel blue
            'green': "\033[38;2;60;179;113m",     # Medium sea green
            'yellow': "\033[38;2;204;204;0m",     # Olive-like dark yellow
            'purple': "\033[38;2;147;112;219m",   # Medium purple
            'cyan': "\033[38;2;0;139;139m"        # Dark cyan
        }


color_code = color_codes.get(thinking_color.lower(), color_codes['green'])   
def show_thinking_header():
    """Display beautiful thinking header"""
    if show_thinking:
        print(f"\n{color_code}╭─────────────────────── 🤔 Thinking ───────────────────────╮{RESET}")

def show_thinking_footer():
    """Display beautiful thinking footer"""
    if show_thinking:
        print(f"{color_code}╰────────────────────────────────────────────────────────────╯{RESET}")

In [4]:
# | export
class ResponseProcessor:
    """Handles processing of LLM responses"""
    
    def __init__(self):
        self.console = Console()
    
    def process_response(self, response: Any, console: Optional[Console] = None) -> Dict[str, Any]:
        """Process non-streaming response"""
        if console is None:
            console = self.console
        
        try:
            message = response.choices[0].message
            
            cleaned_content = ""
            if hasattr(message, 'reasoning') and message.reasoning:
                show_thinking_header()
                console.print(Markdown(message.reasoning))
                show_thinking_footer()
            if hasattr(message, 'content') and message.content:
                content = message.content
        
                # Extract <think>...</think> content
                think_matches = re.findall(r"<think>(.*?)</think>", content, re.DOTALL)
        
                # Remove <think>...</think> blocks from original content
                cleaned_content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
    
                # Print each <think> block inside the fancy box
                for think in think_matches:
                    think = think.strip()
                    if think:
                        show_thinking_header()
                        print(f"{color_code}{think}{RESET}")
                        show_thinking_footer()
        
                # Print main message content
                if cleaned_content.strip():
                    console.print(Markdown(cleaned_content.strip()))
            
            # Extract tool calls
            tool_calls = []
            if hasattr(message, 'tool_calls') and message.tool_calls:
                tool_calls = self._extract_tool_calls(message.tool_calls)
            
            return {
                "content": cleaned_content,
                "tool_calls": tool_calls,
                "usage": getattr(response, 'usage', None),
                "model": getattr(response, 'model', None)
            }
            
        except Exception as e:
            console.print(f"[red]Error processing response: {e}[/red]")
            return {"content": "", "tool_calls": [], "error": str(e)}
    
    def _extract_tool_calls(self, tool_calls) -> list:
        """Extract tool calls from response"""
        extracted_calls = []
        
        for tool_call in tool_calls:
            try:
                extracted_call = {
                    "id": tool_call.id,
                    "type": tool_call.type,
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                }
                extracted_calls.append(extracted_call)
            except Exception as e:
                print(f"Error extracting tool call: {e}")
                continue
        
        return extracted_calls
